<a href="https://colab.research.google.com/github/CansecoSergio/python-data-science/blob/main/M5%20Enunciados%20Practica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# instalar streamlit
!pip install -q streamlit

# bajar ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar xvf ngrok-v3-stable-linux-amd64.tgz

# guardar token
!./ngrok config add-authtoken 2fnTvA033DsLeciIE8BLJyiZ2It_4cpbKgtpPgRHxSQ7sfhWM

# levantar tunel
!./ngrok http 8501 &>/content/ngrok.log &

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
--2024-05-09 15:21:51--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8949853 (8.5M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz’

ngrok-v3-stable-lin 100%[===================>]   8.54M  14.0MB/s    in 0.6s    

2024-05-09 15:21:52 (14.0 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz’ saved [8949853/8949853]

ngrok
Authtok

In [ ]:
%%writefile app.py
"""
Dashboard

Requiere streamlit y pandas y numpy, etc

"""

import streamlit as st
import pandas as pd

st.title("Tablero de bicicletas de NY")

# Funcion con cache
@st.cache_data(ttl= "1d")
def load_bikes_data():
  data = pd.read_csv("/content/drive/MyDrive/Archivos Collab/M5/202109-citibike-tripdata_small.csv")
  data['started_at'] = pd.to_datetime(data['started_at'])
  data['ended_at'] = pd.to_datetime(data['ended_at'])
  return data

bike_data = load_bikes_data()

# Mostrar datos crudos
show_all = st.sidebar.checkbox("Show raw data")

# Mostrar datos por hora
show_by_hour = st.sidebar.checkbox("Show hour chart")

# Slider de horas para mapa
hours_slider = st.sidebar.slider("Hours",min_value=0,max_value=23,step=1)

if show_all:
  st.dataframe(bike_data)

filtered_data = bike_data[bike_data['started_at'].dt.hour <= hours_slider]
filtered_data = filtered_data.rename(columns = {'start_lat':'lat','start_lng':'lon'})

if show_by_hour:
  data_by_hour = pd.DataFrame(filtered_data.groupby(filtered_data['started_at'].dt.hour).count()['started_at'])
  data_by_hour = data_by_hour.rename(columns={'started_at':"total"})
  st.bar_chart(data_by_hour, y='total')


#st.map(filtered_data, latitude="lat", longitude="lon", use_container_width=True)

group_by_latlon = filtered_data.groupby([filtered_data['lat'],filtered_data['lon']], as_index=False).count()[['lon','lat','ride_id']]
group_by_latlon['size'] = group_by_latlon['ride_id']*10

st.map(group_by_latlon, latitude="lat", longitude="lon", use_container_width=True, size='size')


Writing app.py


In [ ]:
!streamlit run /content/app.py &>/content/streamlit.log &

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c 'import sys, json; print("Execute the next cell and then go to the following URL: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

Execute the next cell and then go to the following URL: https://d4c1-34-127-59-181.ngrok-free.app


In [ ]:
import pandas as pd

data = pd.read_csv("/content/202109-citibike-tripdata_small.csv")
data['started_at'] = pd.to_datetime(data['started_at'])
data['ended_at'] = pd.to_datetime(data['ended_at'])


In [ ]:
grouped = data.groupby([data['start_lng'],data['start_lat']], as_index=False).count()[['start_lng','start_lat','ride_id']]
grouped


,start_lng,start_lat,ride_id
0,-74.024714,40.638246,1
1,-74.020005,40.635679,2
2,-74.016777,40.705693,1
3,-74.016584,40.715338,22
4,-74.014847,40.711444,87
...,...,...,...
293,-73.891050,40.861748,8
294,-73.890745,40.768728,2
295,-73.888456,40.868487,2
296,-73.888390,40.871420,2
